In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanza

In [12]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [15]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<div id="remository">20</div>')
tag=soup.div
print(tag.string)

20


In [20]:
soup = BeautifulSoup('<div><span style="background-color:rgba(242, 136, 136,0.075);">then</span> <span style="background-color:rgba(242, 136, 136,0.0625);">a</span> <span style="background-color:rgba(242, 136, 136,0.0375);">few</span> <span style="background-color:rgba(242, 136, 136,0.125);">days</span> <span style="background-color:rgba(242, 136, 136,0.125);">later</span> <span style="background-color:rgba(242, 136, 136,0.17500000000000002);">tell</span> <span style="background-color:rgba(242, 136, 136,0.075);">me</span> <span style="background-color:rgba(242, 136, 136,0.26249999999999996);">the</span> <span style="background-color:rgba(242, 136, 136,1.25);">price</span> <span style="background-color:rgba(242, 136, 136,0.5);">was</span> <span style="background-color:rgba(242, 136, 136,0.44999999999999996);">too</span> <span style="background-color:rgba(242, 136, 136,1.25);">low</span> <span style="background-color:rgba(242, 136, 136,0.3375);">and</span> <span style="background-color:rgba(242, 136, 136,0.1625);">ask</span> <span style="background-color:rgba(242, 136, 136,0.13749999999999998);">for</span> <span style="background-color:rgba(242, 136, 136,0.125);">more</span> <span style="background-color:rgba(242, 136, 136,0.125);">.</span></div>')

In [27]:
len(' '.join([tag.string for tag in soup.find_all('span')]))

70

In [22]:
tag.string

'then'

In [10]:
1232 // 127

9

In [2]:
combined = pd.read_csv('combined_reviews_eng.csv')

In [3]:
from nltk import tokenize

In [4]:
combined['content'] = combined.content.str.replace('\n', '. ') # quick little cleaning step 
# bol['content'] = bol.content.str.replace('. .', '. ', regex=False) # quick little cleaning step 

combined['sentences'] = combined['content'].apply(lambda x: tokenize.sent_tokenize(x))

In [7]:
combined = combined.explode('sentences').reset_index(drop=True)

In [8]:
combined = combined[(combined['sentences'].str.len() > 15)].reset_index(drop=True)

In [9]:
combined.to_csv('combined_reviews_eng_CLEAN.csv')

In [16]:
combined[(combined['company'] == 'booking.com') & (
    combined['sentences'].str.contains('service'))]

,Unnamed: 0,name,nb_reviews,location,rating,header,content,date,responded,response_date,response,company,sentences
2,0,Alexander Santos,1,US,1,Terrible experience with this website,Terrible experience with this website. I made ...,2021-04-19 19:26:11+00:00,0,NaN,-,booking.com,"Customer service is awful, I have spent three ..."
55,9,Roberta,3,GB,1,No customer service…,Had a hotel booking in 2020 which was paid on ...,2021-04-14 15:45:28+00:00,0,NaN,-,booking.com,Awful customer service.
64,11,Rotem,1,IL,2,Don't book Isrotel hotel rooms on booking.com,I used to use booking.com for hotel reservatio...,2021-04-13 11:17:23+00:00,0,NaN,-,booking.com,They finally agreed to do the change but were ...
88,17,Daniel Mardasy,5,NL,1,Booked a room through their website,Booked a room through their website. When we w...,2021-04-11 16:48:28+00:00,0,NaN,-,booking.com,We've called booking.com but it took us almost...
91,17,Daniel Mardasy,5,NL,1,Booked a room through their website,Booked a room through their website. When we w...,2021-04-11 16:48:28+00:00,0,NaN,-,booking.com,An awful experience and an unexpected service ...
116,20,Alexander Santos,1,US,1,Terrible experience with this website,Terrible experience with this website. I made ...,2021-04-19 19:26:11+00:00,0,NaN,-,booking.com,"Customer service is awful, I have spent three ..."
169,29,Roberta,3,GB,1,No customer service…,Had a hotel booking in 2020 which was paid on ...,2021-04-14 15:45:28+00:00,0,NaN,-,booking.com,Awful customer service.
178,31,Rotem,1,IL,2,Don't book Isrotel hotel rooms on booking.com,I used to use booking.com for hotel reservatio...,2021-04-13 11:17:23+00:00,0,NaN,-,booking.com,They finally agreed to do the change but were ...
202,37,Daniel Mardasy,5,NL,1,Booked a room through their website,Booked a room through their website. When we w...,2021-04-11 16:48:28+00:00,0,NaN,-,booking.com,We've called booking.com but it took us almost...
205,37,Daniel Mardasy,5,NL,1,Booked a room through their website,Booked a room through their website. When we w...,2021-04-11 16:48:28+00:00,0,NaN,-,booking.com,An awful experience and an unexpected service ...


In [8]:
customer_service_df = combined[combined['sentences'].str.contains('customer service')]

In [9]:
import aspect_based_sentiment_analysis as absa
recognizer = absa.aux_models.BasicPatternRecognizer()
NLP = absa.load(pattern_recognizer=recognizer)

In [10]:
from tqdm import tqdm

examples = []

for sentence in tqdm(customer_service_df['sentences']):
    
    run = NLP(text=sentence, aspects=['customer service'])
    
    examples.append(run.examples[0])
    


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:09<00:00,  2.52it/s]


In [11]:
import numpy as np

In [12]:
aspect = 'customer service'

# Average distribution (neutral | negative | positive)
avg_distribution = np.sum(np.array([example.scores for example in examples]), axis=0) / len(examples)

print(f'Reviews mentioning {aspect}')
avg_distribution

Reviews mentioning customer service


array([0.05962507, 0.7127558 , 0.22761922], dtype=float32)

In [38]:
sentiment_scores = [example.scores[2] - example.scores[1] for idx, example in enumerate(examples)]

In [82]:
sentiment_scores

[0.99726415,
 0.99751806,
 -0.9805648,
 -0.71340114,
 -0.9855761,
 -0.9879231,
 0.9974816,
 0.99731296,
 -0.9870893,
 -0.9166326,
 -0.5877112,
 0.9974704,
 -0.98729986,
 -0.15822676,
 -0.9850764,
 -0.9873056,
 -0.6892363,
 -0.6380442,
 -0.9784847,
 -0.96192706,
 -0.95916384,
 -0.96119434,
 -0.9751521,
 -0.6888849,
 -0.98656595]

In [98]:
fig = px.histogram(sentiment_scores, nbins=25)

fig.update_layout(
    xaxis_title_text = 'Sentiment',
    yaxis_title_text = 'Count',
    bargap=0.2,
    showlegend=False,
    dragmode=False,
    clickmode='none',
)


# fig.update_traces(hovertemplate=None)
fig.update_traces(hovertemplate=
                  '<i><b>Count</i>: %{y}' + 
                  '<br><b>Sentiment Range</b>: %{x}' +
                  '<br><extra></extra>')



fig.show()

In [87]:
sentiment_scores

[0.99726415,
 0.99751806,
 -0.9805648,
 -0.71340114,
 -0.9855761,
 -0.9879231,
 0.9974816,
 0.99731296,
 -0.9870893,
 -0.9166326,
 -0.5877112,
 0.9974704,
 -0.98729986,
 -0.15822676,
 -0.9850764,
 -0.9873056,
 -0.6892363,
 -0.6380442,
 -0.9784847,
 -0.96192706,
 -0.95916384,
 -0.96119434,
 -0.9751521,
 -0.6888849,
 -0.98656595]

In [15]:
pos_scores.argsort()[-5:][::-1]

array([11,  6,  1,  7,  0], dtype=int64)

In [14]:
# positive scores
pos_scores = np.array([example.scores[2] for idx, example in enumerate(examples)])
pos_scores

array([0.99834883, 0.99845755, 0.0055188 , 0.10376485, 0.00646426,
       0.00528512, 0.99850106, 0.9984359 , 0.00569052, 0.01087566,
       0.14215325, 0.99853694, 0.00552637, 0.04802295, 0.00426602,
       0.005228  , 0.14547388, 0.04720908, 0.01017662, 0.00662997,
       0.00800654, 0.01455655, 0.0069249 , 0.11073714, 0.00568936],
      dtype=float32)

In [24]:
best_ordered = np.array([example.scores[2] for idx, example in enumerate(examples)]).argsort()[::-1]

In [28]:
[score for score in pos_scores[best_ordered] if score > 0.6][:5]

[0.99853694, 0.99850106, 0.99845755, 0.9984359, 0.99834883]

In [23]:
pos_scores

array([0.99834883, 0.99845755, 0.0055188 , 0.10376485, 0.00646426,
       0.00528512, 0.99850106, 0.9984359 , 0.00569052, 0.01087566,
       0.14215325, 0.99853694, 0.00552637, 0.04802295, 0.00426602,
       0.005228  , 0.14547388, 0.04720908, 0.01017662, 0.00662997,
       0.00800654, 0.01455655, 0.0069249 , 0.11073714, 0.00568936],
      dtype=float32)

In [19]:
# negative scores
np.array([example.scores[1] for idx, example in enumerate(examples)])

array([1.0846786e-03, 9.3948544e-04, 9.8608357e-01, 8.1716597e-01,
       9.9204034e-01, 9.9320823e-01, 1.0194661e-03, 1.1229619e-03,
       9.9277979e-01, 9.2750823e-01, 7.2986448e-01, 1.0665880e-03,
       9.9282622e-01, 2.0624970e-01, 9.8934245e-01, 9.9253356e-01,
       8.3471018e-01, 6.8525326e-01, 9.8866129e-01, 9.6855700e-01,
       9.6717036e-01, 9.7575086e-01, 9.8207700e-01, 7.9962206e-01,
       9.9225533e-01], dtype=float32)

In [17]:
[example.scores for idx, example in enumerate(examples)]

[[0.0005665521, 0.0010846786, 0.99834883],
 [0.0006029623, 0.00093948544, 0.99845755],
 [0.008397679, 0.98608357, 0.005518797],
 [0.0790692, 0.817166, 0.10376485],
 [0.0014954025, 0.99204034, 0.0064642555],
 [0.0015066285, 0.9932082, 0.0052851154],
 [0.00047944364, 0.0010194661, 0.99850106],
 [0.0004410581, 0.0011229619, 0.9984359],
 [0.0015296724, 0.9927798, 0.005690519],
 [0.061616074, 0.92750823, 0.01087566],
 [0.12798229, 0.7298645, 0.14215325],
 [0.00039649796, 0.001066588, 0.99853694],
 [0.0016474467, 0.9928262, 0.0055263657],
 [0.74572736, 0.2062497, 0.04802295],
 [0.0063915257, 0.98934245, 0.0042660222],
 [0.0022383155, 0.99253356, 0.005228005],
 [0.019815892, 0.8347102, 0.14547388],
 [0.26753768, 0.68525326, 0.047209077],
 [0.0011621026, 0.9886613, 0.010176622],
 [0.024813004, 0.968557, 0.0066299695],
 [0.024823045, 0.96717036, 0.008006539],
 [0.009692586, 0.97575086, 0.014556548],
 [0.010998204, 0.982077, 0.006924901],
 [0.08964077, 0.79962206, 0.11073714],
 [0.0020553458, 0.

In [46]:
examples[0].scores

[0.0005665521, 0.0010846786, 0.99834883]

In [21]:
for idx, _ in enumerate(examples):

    absa.summary(examples[idx])

    absa.display(examples[idx].review)

Sentiment.positive for "customer service"
Scores (neutral/negative/positive): [0.001 0.001 0.998]


Sentiment.positive for "customer service"
Scores (neutral/negative/positive): [0.001 0.001 0.998]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.008 0.986 0.006]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.079 0.817 0.104]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.001 0.992 0.006]


...

Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.025 0.967 0.008]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.01  0.976 0.015]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.011 0.982 0.007]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.09  0.8   0.111]


Sentiment.negative for "customer service"
Scores (neutral/negative/positive): [0.002 0.992 0.006]


### FIRST EXAMPLE

In [35]:
import aspect_based_sentiment_analysis as absa

recognizer = absa.aux_models.BasicPatternRecognizer()
NLP = absa.load(pattern_recognizer=recognizer)

print('Text to execute ABSA on:')
print(bol['content'][4])

test_run = NLP(text=bol['content'][4], aspects=['customer', 'price', 'delivery'])

customer, price, delivery = test_run.examples

In [36]:
absa.summary(customer)
absa.display(customer.review)

Sentiment.positive for "customer"
Scores (neutral/negative/positive): [0.001 0.001 0.998]


In [28]:
absa.summary(price)
absa.display(price.review)

Sentiment.positive for "price"
Scores (neutral/negative/positive): [0.162 0.336 0.502]


In [29]:
absa.summary(delivery)
absa.display(delivery.review)

Sentiment.negative for "delivery"
Scores (neutral/negative/positive): [0.161 0.625 0.214]


### OLD CODE

In [9]:
# Make sure you have downloaded the StanfordNLP English model and other essential tools using,
# stanfordnlp.download('en')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def aspect_sentiment_analysis(txt, stop_words, nlp):
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)

        doc = nlp(finaltxt) # Object of Stanford NLP Pipeleine
        
        # Getting the dependency relations betwwen the words
        dep_node = []

        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])

        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
                dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]

        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
        
    return finalcluster

nlp = stanza.Pipeline('en')
stop_words = set(stopwords.words('english'))
# txt = "De muziek kwaliteit is erg goed maar de levensduur van de batterij is verschrikkelijk."
txt = "This is an super interesting testrun that has gone horribly wrong."


test = aspect_sentiment_analysis(txt, stop_words, nlp)

2021-04-11 11:46:35 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-04-11 11:46:35 INFO: Use device: gpu
2021-04-11 11:46:35 INFO: Loading: tokenize
2021-04-11 11:46:36 INFO: Loading: pos
2021-04-11 11:46:36 INFO: Loading: lemma
2021-04-11 11:46:37 INFO: Loading: depparse
2021-04-11 11:46:37 INFO: Loading: sentiment
2021-04-11 11:46:37 INFO: Loading: ner
2021-04-11 11:46:38 INFO: Done loading processors!
